In [ ]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

# movie.csv와 user, item, cluster, tsne좌표로 이루어진 데이터프레임을 생성합니다.
# 파일 경로와 실제 데이터에 맞게 코드를 수정해야 합니다.
data_dir = "/data/ephemeral/level2-movierecommendation-recsys-06/data/train/"
cluster_3d = "kmeans3d"
movie_df = pd.read_csv(data_dir + f"movie_{cluster_3d}.csv")
user_movie_df = pd.read_csv(data_dir + f"user_movie_{cluster_3d}.csv")

In [ ]:
# 각 유저의 상위 10개 클러스터를 추출합니다.
user_top_clusters = pd.Series(
    user_movie_df.groupby(["user", "cluster"])
    .size()
    .groupby("user")
    .nlargest(10)
    .reset_index(level=1, drop=True)
    , name="count"
).to_frame()
user_top_clusters

In [ ]:
# "user"와 "cluster"를 기준으로 그룹화하고 T-SNE1 및 T-SNE2의 평균 계산
avg_tsne_values = user_movie_df.groupby(["user", "cluster"])[["t-SNE1", "t-SNE2", "t-SNE3"]].mean().reset_index()

# user_movie_df와 avg_tsne_values를 user 및 cluster를 기준으로 조인
user_movie_df_with_avg_tsne = pd.merge(user_top_clusters, avg_tsne_values, on=["user", "cluster"], how="left")

user_movie_df_with_avg_tsne

In [ ]:
cluster = defaultdict(dict)
for i, ser in movie_df.iterrows():
    cluster[ser["cluster"]][ser["item"]] = 0
cluster

In [ ]:
user = defaultdict(list)
for i, ser in user_movie_df.iterrows():
    user[ser["user"]].append(ser["item"])
    cluster[ser["cluster"]][ser["item"]] += 1
user

In [ ]:
movie = movie_df.set_index("item")
movie.loc[10]

In [ ]:
dic = {"user": [], "item": []}

for i, avg in tqdm(user_movie_df_with_avg_tsne.iterrows(), total=user_movie_df_with_avg_tsne.shape[0]):
    tmp = []
    for item_id in cluster[avg["cluster"]]:
        dis = (
            (avg["t-SNE1"] - movie.loc[item_id]["t-SNE1"]) ** 2
            + (avg["t-SNE2"] - movie.loc[item_id]["t-SNE2"]) ** 2
            + (avg["t-SNE3"] - movie.loc[item_id]["t-SNE3"]) ** 2
        )
        if item_id not in user[avg["user"]]:
            if len(tmp) != 2:
                tmp.append([item_id, dis])
                tmp.sort(key=lambda x: x[1])
            else:
                if tmp[0][1] > dis:
                    tmp[1] = [item_id, dis]
                    tmp.sort(key=lambda x: x[1])
                elif tmp[0][1] == dis or tmp[1][1] >= dis:
                    tmp[1] = [item_id, dis]
    dic["user"].append(avg["user"])
    dic["item"].append(tmp[0][0])
    dic["user"].append(avg["user"])
    dic["item"].append(tmp[1][0])
dic

In [ ]:
submission = pd.DataFrame.from_dict(data=dic, orient="columns")
submission = submission.astype(int)
submission

In [ ]:
submission.to_csv("../data/eval/submission_3d.csv", index=False)